In [17]:
import os
import glob
from collections import Counter
import pandas as pd

# 데이터 경로
data_dir = "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721"

# jpg, txt 파일 리스트 수집 (classes.txt 제외)
jpg_files = glob.glob(os.path.join(data_dir, "*.jpg")) + glob.glob(os.path.join(data_dir, "*.JPG"))
txt_files = [f for f in glob.glob(os.path.join(data_dir, "*.txt")) if os.path.basename(f) != "classes.txt"]

# 클래스 카운트 (YOLO txt 포맷: class x y w h ...)
class_counts = Counter()
for txt in txt_files:
    with open(txt, "r") as f:
        for line in f:
            if line.strip():
                cls = line.split()[0]   # 첫 번째 값이 class index
                class_counts[cls] += 1

# DataFrame으로 보기 좋게 출력
summary = pd.DataFrame.from_dict(class_counts, orient="index", columns=["count"])
summary.loc["TOTAL"] = summary["count"].sum()

print(f"📸 JPG 파일 개수: {len(jpg_files)}")
print(f"📄 TXT 파일 개수 (classes.txt 제외): {len(txt_files)}")
display(summary)



📸 JPG 파일 개수: 3511
📄 TXT 파일 개수 (classes.txt 제외): 2794


,count
1,3477
0,1362
2,248
TOTAL,5087


In [18]:
import os
import glob
from collections import Counter
import pandas as pd

# 데이터 경로
data_dir = "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725"

# jpg, txt 파일 리스트 수집 (소문자/대문자 모두 대응)
jpg_files = glob.glob(os.path.join(data_dir, "*.jpg")) + glob.glob(os.path.join(data_dir, "*.JPG"))
txt_files = glob.glob(os.path.join(data_dir, "*.txt"))

# 클래스 카운트 (YOLO txt 포맷: class x y w h ...)
class_counts = Counter()
for txt in txt_files:
    with open(txt, "r") as f:
        for line in f:
            if line.strip():
                cls = line.split()[0]   # 첫 번째 값이 class index
                class_counts[cls] += 1

# DataFrame으로 보기 좋게 출력
summary = pd.DataFrame.from_dict(class_counts, orient="index", columns=["count"])
summary.loc["TOTAL"] = summary["count"].sum()

print(f"📸 JPG 파일 개수: {len(jpg_files)}")
print(f"📄 TXT 파일 개수: {len(txt_files)}")
display(summary)


📸 JPG 파일 개수: 1688
📄 TXT 파일 개수: 1348


,count
1,1389
0,1528
2,62
TOTAL,2979


In [5]:
# ipynb 셀: Data_20250721에서 "bad 이미지" 자동 탐지 후 이미지+라벨 동시 삭제

import os
from pathlib import Path
from PIL import Image, UnidentifiedImageError

DATA_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721")
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp", ".JPG", ".JPEG", ".PNG", ".BMP", ".TIF", ".TIFF", ".WEBP"}

MIN_BYTES_SUSPECT = 1024   # 1KB 미만이면 손상 의심
DRY_RUN = False            # ← True면 '예행연습(미삭제)', False면 실제 삭제

def is_bad_image(p: Path):
    """열기/검증 실패 또는 너무 작은 파일이면 bad로 판단. (확장자-포맷 불일치는 삭제 사유로 간주하지 않음)"""
    try:
        size_b = os.path.getsize(p)
    except Exception:
        return True, "stat_error"

    if size_b < MIN_BYTES_SUSPECT:
        return True, "tiny_file"

    try:
        with Image.open(p) as im:
            im.verify()   # 헤더/트레일러 검증
        # 여기까지 오면 정상. (포맷 불일치는 삭제하지 않음)
        return False, None
    except UnidentifiedImageError:
        return True, "unidentified_image"
    except Exception as e:
        return True, f"error:{type(e).__name__}"

deleted_imgs, deleted_lbls = 0, 0
bad_list = []

for p in DATA_DIR.iterdir():  # 하위 폴더가 있으면 .rglob("*")로 바꿔도 됨
    if not p.is_file() or p.suffix not in IMG_EXTS:
        continue
    bad, reason = is_bad_image(p)
    if bad:
        bad_list.append((str(p), reason))
        # 라벨 경로(동일 stem의 .txt)
        lbl = p.with_suffix(".txt")
        if not DRY_RUN:
            try:
                p.unlink()
                deleted_imgs += 1
            except Exception as e:
                print(f"[WARN] 이미지 삭제 실패: {p} ({e})")
            if lbl.exists():
                try:
                    lbl.unlink()
                    deleted_lbls += 1
                except Exception as e:
                    print(f"[WARN] 라벨 삭제 실패: {lbl} ({e})")

print("=== BAD IMAGE CLEANUP SUMMARY ===")
print(f"대상 폴더: {DATA_DIR}")
print(f"탐지된 bad 이미지 수: {len(bad_list)}")
print(f"삭제된 이미지: {deleted_imgs}  / 삭제된 라벨: {deleted_lbls}")
if DRY_RUN:
    print("※ DRY_RUN=True 이므로 실제 삭제는 수행하지 않았습니다.")

# 필요 시 어떤 파일이 지워졌는지 목록 출력(상위 30개만)
for i, (pp, rsn) in enumerate(bad_list[:30], 1):
    print(f"{i:02d}. {rsn} -> {pp}")

# 전체 목록을 텍스트 로그로 남기고 싶으면 아래 주석 해제
# log_path = DATA_DIR / "_bad_deleted.log"
# with open(log_path, "w", encoding="utf-8") as f:
#     for pp, rsn in bad_list:
#         f.write(f"{rsn}\t{pp}\n")
# print("로그 저장:", log_path)



=== BAD IMAGE CLEANUP SUMMARY ===
대상 폴더: /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721
탐지된 bad 이미지 수: 425
삭제된 이미지: 425  / 삭제된 라벨: 0
01. tiny_file -> /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721/20250721_053640_0036_000142_f001410_1920x1536_5515.jpg
02. tiny_file -> /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721/20250721_053640_0036_000089_f000880_1920x1536_5388.jpg
03. tiny_file -> /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721/20250721_053640_0034_000025_f000240_1920x1536_4280.jpg
04. tiny_file -> /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721/20250721_053640_0032_000074_f000730_1920x1536_3924.jpg
05. tiny_file -> /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721/20250721_053640_0035_000145_f001440_1920x1536_5055.jpg
06. tiny_file -> /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721/20250721_053640_0032_000153_f001520_1920x1536_4145.jpg
07. tiny_file -> /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721/20250721_053640_0

In [6]:
# ipynb 셀: Data_20250721에서 "bad 이미지" 자동 탐지 후 이미지+라벨 동시 삭제

import os
from pathlib import Path
from PIL import Image, UnidentifiedImageError

DATA_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725")
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp", ".JPG", ".JPEG", ".PNG", ".BMP", ".TIF", ".TIFF", ".WEBP"}

MIN_BYTES_SUSPECT = 1024   # 1KB 미만이면 손상 의심
DRY_RUN = False            # ← True면 '예행연습(미삭제)', False면 실제 삭제

def is_bad_image(p: Path):
    """열기/검증 실패 또는 너무 작은 파일이면 bad로 판단. (확장자-포맷 불일치는 삭제 사유로 간주하지 않음)"""
    try:
        size_b = os.path.getsize(p)
    except Exception:
        return True, "stat_error"

    if size_b < MIN_BYTES_SUSPECT:
        return True, "tiny_file"

    try:
        with Image.open(p) as im:
            im.verify()   # 헤더/트레일러 검증
        # 여기까지 오면 정상. (포맷 불일치는 삭제하지 않음)
        return False, None
    except UnidentifiedImageError:
        return True, "unidentified_image"
    except Exception as e:
        return True, f"error:{type(e).__name__}"

deleted_imgs, deleted_lbls = 0, 0
bad_list = []

for p in DATA_DIR.iterdir():  # 하위 폴더가 있으면 .rglob("*")로 바꿔도 됨
    if not p.is_file() or p.suffix not in IMG_EXTS:
        continue
    bad, reason = is_bad_image(p)
    if bad:
        bad_list.append((str(p), reason))
        # 라벨 경로(동일 stem의 .txt)
        lbl = p.with_suffix(".txt")
        if not DRY_RUN:
            try:
                p.unlink()
                deleted_imgs += 1
            except Exception as e:
                print(f"[WARN] 이미지 삭제 실패: {p} ({e})")
            if lbl.exists():
                try:
                    lbl.unlink()
                    deleted_lbls += 1
                except Exception as e:
                    print(f"[WARN] 라벨 삭제 실패: {lbl} ({e})")

print("=== BAD IMAGE CLEANUP SUMMARY ===")
print(f"대상 폴더: {DATA_DIR}")
print(f"탐지된 bad 이미지 수: {len(bad_list)}")
print(f"삭제된 이미지: {deleted_imgs}  / 삭제된 라벨: {deleted_lbls}")
if DRY_RUN:
    print("※ DRY_RUN=True 이므로 실제 삭제는 수행하지 않았습니다.")

# 필요 시 어떤 파일이 지워졌는지 목록 출력(상위 30개만)
for i, (pp, rsn) in enumerate(bad_list[:30], 1):
    print(f"{i:02d}. {rsn} -> {pp}")

# 전체 목록을 텍스트 로그로 남기고 싶으면 아래 주석 해제
# log_path = DATA_DIR / "_bad_deleted.log"
# with open(log_path, "w", encoding="utf-8") as f:
#     for pp, rsn in bad_list:
#         f.write(f"{rsn}\t{pp}\n")
# print("로그 저장:", log_path)



=== BAD IMAGE CLEANUP SUMMARY ===
대상 폴더: /home/dw/ws_job_msislab/Golf_Project/data/Data_20250725
탐지된 bad 이미지 수: 0
삭제된 이미지: 0  / 삭제된 라벨: 0


In [7]:
# ipynb 셀: 두 폴더 병합 + YOLOv8용 train/val/test 분할 + 요약 출력
import os, shutil, glob, random, json
from collections import Counter, defaultdict
from pathlib import Path

# ====== 설정 ======
# 병합할 원본 폴더들
SRC_DIRS = [
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721",
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725",
]

# 결과 폴더 (새로 생성)
DEST_DIR = "/home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8"

# 분할 비율
RATIO = (0.8, 0.1, 0.1)  # train, val, test
SEED = 42

# 허용 이미지 확장자
IMG_EXTS = {".jpg", ".JPG", ".jpeg", ".JPEG", ".png", ".PNG"}

# ====== 유틸 ======
def is_anno_file(p: Path) -> bool:
    # classes.txt 같은 정의 파일 제외, 순수 라벨 파일만 인정
    return p.suffix == ".txt" and p.name != "classes.txt"

def read_label_lines(label_path: Path):
    if not label_path.exists():
        return []
    with open(label_path, "r", encoding="utf-8") as f:
        return [ln.strip() for ln in f if ln.strip()]

def parse_classes_from_lines(lines):
    classes = []
    for ln in lines:
        parts = ln.split()
        if not parts:
            continue
        # YOLO: class x y w h ...
        cls = parts[0]
        # 숫자만 카운트
        if cls.isdigit():
            classes.append(int(cls))
    return classes

def ensure_empty_label(path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    if not path.exists():
        with open(path, "w", encoding="utf-8") as f:
            pass  # empty file

def copy_pair(src_img: Path, src_lbl: Path | None, dst_img: Path, dst_lbl: Path):
    dst_img.parent.mkdir(parents=True, exist_ok=True)
    dst_lbl.parent.mkdir(parents=True, exist_ok=True)
    shutil.copy2(src_img, dst_img)
    if src_lbl and src_lbl.exists():
        shutil.copy2(src_lbl, dst_lbl)
    else:
        ensure_empty_label(dst_lbl)

def choose_primary_class(classes: list[int]) -> str:
    if not classes:
        return "none"
    cnt = Counter(classes)
    # 가장 많은 클래스, 동률이면 id가 작은 것
    max_freq = max(cnt.values())
    tied = [c for c, v in cnt.items() if v == max_freq]
    return str(min(tied))

def yolo_summary(label_files: list[Path]):
    obj_counter = Counter()
    for lf in label_files:
        lines = read_label_lines(lf)
        for ln in lines:
            parts = ln.split()
            if parts and parts[0].isdigit():
                obj_counter[parts[0]] += 1
    total = sum(obj_counter.values())
    return obj_counter, total

# ====== 원본 스캔: 이미지-라벨 페어 수집 ======
pairs = []  # (src_img, src_lbl, src_tag)
for src in SRC_DIRS:
    tag = Path(src).name  # 예: Data_20250721
    for img_path in Path(src).rglob("*"):
        if img_path.suffix in IMG_EXTS and img_path.is_file():
            stem = img_path.stem
            # 라벨 파일은 동일 경로/동일 스템의 .txt로 가정
            lbl_path = img_path.with_suffix(".txt")
            # classes.txt 같은 잡파일 제외
            if lbl_path.name == "classes.txt":
                lbl_path = Path("__NO_LABEL__")  # 존재하지 않는 경로로 처리
            pairs.append((img_path, lbl_path if lbl_path.exists() else None, tag))

print(f"✅ 원본에서 수집된 이미지 수: {len(pairs)}")

# ====== 주 클래스 기준 버킷팅 ======
buckets = defaultdict(list)  # key: primary_class, value: list of entries
entries = []
for img_path, lbl_path, tag in pairs:
    lines = read_label_lines(lbl_path) if lbl_path else []
    classes = parse_classes_from_lines(lines)
    pcls = choose_primary_class(classes)  # '0','1','2' 또는 'none'
    entries.append((img_path, lbl_path, tag, pcls))

# 클래스별로 섞고 분할 비율대로 나누기
random.seed(SEED)
for pcls in set(e[3] for e in entries):
    cls_group = [e for e in entries if e[3] == pcls]
    random.shuffle(cls_group)
    n = len(cls_group)
    n_tr = int(n * RATIO[0])
    n_vl = int(n * RATIO[1])
    # 남은 건 test
    n_ts = n - n_tr - n_vl
    buckets[pcls] = [
        ("train", cls_group[:n_tr]),
        ("val",   cls_group[n_tr:n_tr+n_vl]),
        ("test",  cls_group[n_tr+n_vl:]),
    ]

# ====== 목적지 디렉토리 구조 생성 ======
for split in ["train", "val", "test"]:
    (Path(DEST_DIR)/"images"/split).mkdir(parents=True, exist_ok=True)
    (Path(DEST_DIR)/"labels"/split).mkdir(parents=True, exist_ok=True)

# ====== 복사(병합) 진행 ======
def build_new_name(tag: str, src_img: Path):
    # 파일명 충돌 방지: {원본폴더명}_{원본파일명}.{ext}
    return f"{tag}_{src_img.stem}{src_img.suffix}"

copied = {"train": 0, "val": 0, "test": 0}
for pcls, splits in buckets.items():
    for split_name, items in splits:
        for (img_path, lbl_path, tag, _) in items:
            new_img_name = build_new_name(tag, img_path)
            dst_img = Path(DEST_DIR)/"images"/split_name/new_img_name
            dst_lbl = (Path(DEST_DIR)/"labels"/split_name/new_img_name).with_suffix(".txt")
            copy_pair(img_path, lbl_path, dst_img, dst_lbl)
            copied[split_name] += 1

print(f"📦 병합 및 분할 완료 → {DEST_DIR}")
print("  - train:", copied["train"])
print("  - val  :", copied["val"])
print("  - test :", copied["test"])

# ====== 분할 결과 요약 (이미지/라벨 파일 수 & 클래스별 객체 수) ======
def list_files(split):
    imgs = list((Path(DEST_DIR)/"images"/split).glob("*"))
    lbls = list((Path(DEST_DIR)/"labels"/split).glob("*.txt"))
    return imgs, lbls

def print_split_summary(split):
    imgs, lbls = list_files(split)
    obj_counter, total = yolo_summary(lbls)
    print(f"\n=== {split.upper()} ===")
    print(f"images: {len(imgs)} / labels: {len(lbls)}")
    if obj_counter:
        # class id 정렬
        keys = sorted(obj_counter.keys(), key=lambda x: (x=="none", int(x) if x.isdigit() else 9999))
        print("class\tcount")
        for k in keys:
            print(f"{k}\t{obj_counter[k]}")
        print(f"TOTAL\t{total}")
    else:
        print("(no objects)")

for sp in ["train", "val", "test"]:
    print_split_summary(sp)

# ====== data.yaml 초안 저장 (선택) ======
yaml_path = Path(DEST_DIR)/"data.yaml"
yaml_content = f"""# YOLOv8 dataset config
path: {DEST_DIR}
train: images/train
val: images/val
test: images/test

names:
  0: DIVOT
  1: FIXED_DIVOT
  2: GRASS
"""
with open(yaml_path, "w", encoding="utf-8") as f:
    f.write(yaml_content)
print(f"\n📝 data.yaml 생성: {yaml_path}")


✅ 원본에서 수집된 이미지 수: 4774
📦 병합 및 분할 완료 → /home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8
  - train: 3817
  - val  : 475
  - test : 482

=== TRAIN ===
images: 3817 / labels: 3817
class	count
0	2332
1	3884
2	254
TOTAL	6470

=== VAL ===
images: 475 / labels: 475
class	count
0	272
1	476
2	25
TOTAL	773

=== TEST ===
images: 482 / labels: 482
class	count
0	286
1	506
2	31
TOTAL	823

📝 data.yaml 생성: /home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8/data.yaml


In [8]:
import os, cv2
from PIL import Image

BASE = "/home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8"
IMG_ROOT = os.path.join(BASE, "images")
SPLITS = ["train","val","test"]

bad = []

def is_bad_image(p):
    # 1) 파일 크기 1KB 미만이면 의심
    try:
        if os.path.getsize(p) < 1024:
            return "tiny"
    except: 
        return "stat_error"
    # 2) OpenCV로 시도
    try:
        img = cv2.imdecode(np.fromfile(p, dtype=np.uint8), cv2.IMREAD_COLOR)
        if img is None: 
            return "cv2_fail"
    except Exception:
        return "cv2_exc"
    # 3) PIL로도 시도 (헤더/EXIF 문제 잡힘)
    try:
        with Image.open(p) as im:
            im.verify()
    except Exception:
        return "pil_verify_fail"
    return None

import numpy as np
for split in SPLITS:
    root = os.path.join(IMG_ROOT, split)
    if not os.path.isdir(root): 
        continue
    for f in os.listdir(root):
        if not f.lower().endswith((".jpg",".jpeg",".png",".bmp",".tif",".tiff",".webp")):
            continue
        p = os.path.join(root, f)
        err = is_bad_image(p)
        if err:
            bad.append((split, p, err))

print("BAD count:", len(bad))
for s,p,err in bad[:50]:
    print(s, err, "->", p)

# CSV로 저장
import pandas as pd
out_csv = os.path.join(BASE, "_checks", "bad_images.csv")
os.makedirs(os.path.dirname(out_csv), exist_ok=True)
pd.DataFrame(bad, columns=["split","path","reason"]).to_csv(out_csv, index=False)
print("Saved:", out_csv)


BAD count: 0
Saved: /home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8/_checks/bad_images.csv


In [14]:
import os
from collections import Counter

base_dir = "/home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8/labels"
cls_counter = Counter()

# train/ val/ test/ 모두 탐색
for split in ["train", "val", "test"]:
    label_dir = os.path.join(base_dir, split)
    for file in os.listdir(label_dir):
        if file.endswith(".txt"):
            with open(os.path.join(label_dir, file), "r") as f:
                for line in f:
                    cls_id = int(line.strip().split()[0])  # 라벨 첫 번째 값 = 클래스 인덱스
                    cls_counter[cls_id] += 1

print("클래스별 라벨 개수:")
for cls_id, count in sorted(cls_counter.items()):
    print(f"클래스 {cls_id}: {count}")


클래스별 라벨 개수:
클래스 0: 2890
클래스 1: 4866
클래스 2: 310


# 자 이제 원본 데이터를 가지고 학습을 진행해보자

In [9]:
import torch, platform
print("Python:", platform.python_version())
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version (torch):", torch.version.cuda)
print("Device count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU 0:", torch.cuda.get_device_name(0))
    # 간단 커널 실행(텐서 연산)
    a = torch.randn(1024, 1024, device="cuda")
    b = torch.randn(1024, 1024, device="cuda")
    c = (a @ b).mean().item()
    print("Matmul OK. sample:", c)


Python: 3.10.12
Torch: 2.5.1+cu121
CUDA available: True
CUDA version (torch): 12.1
Device count: 1
GPU 0: NVIDIA GeForce RTX 4060 Ti
Matmul OK. sample: -0.022038821130990982


In [10]:
from ultralytics import YOLO

DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8/data.yaml"

# 모델 선택 (작게는 yolov8n.pt, 기본은 yolov8s.pt)
model = YOLO("yolov8s.pt")

model.train(
    data=DATA_YAML,
    epochs=250, patience=50,
    imgsz=896,
    batch=4,
    device=0,
    workers=8,
    rect=True,
    cache=True,

    # ==== 증강 완전 끔 ====
    mosaic=0.0, mixup=0.0, copy_paste=0.0, erasing=0.0, auto_augment=None, close_mosaic=0,
    hsv_h=0.0, hsv_s=0.0, hsv_v=0.0,
    flipud=0.0, fliplr=0.0,
    degrees=0.0, translate=0.0, scale=0.0, shear=0.0, perspective=0.0,

    # ==== 저장 경로 ====
    project="/home/dw/ws_job_msislab/Golf_Project/runs_yolo",  # 상위 폴더
    name="normal_train",  # 하위 폴더 이름
    exist_ok=True,  # 같은 이름 있으면 덮어씀

    seed=42
)

Ultralytics 8.3.175 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=None, batch=4, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=0, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.0, exist_ok=True, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, imgsz=896, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=normal_train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=50, 

train: Scanning /home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8/labels/train... 3817 images, 505 backgrounds, 0 corrupt: 100%|██████████| 3817/3817 [00:00<00:00, 3868.40it/s]

train: New cache created: /home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8/labels/train.cache


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (6.9GB RAM): 100%|██████████| 3817/3817 [00:09<00:00, 413.82it/s]

...

                   all        475        773      0.794      0.517      0.607      0.517
EarlyStopping: Training stopped early as no improvement observed in last 50 epochs. Best results observed at epoch 176, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



226 epochs completed in 3.933 hours.
Optimizer stripped from /home/dw/ws_job_msislab/Golf_Project/runs_yolo/normal_train/weights/last.pt, 22.6MB
Optimizer stripped from /home/dw/ws_job_msislab/Golf_Project/runs_yolo/normal_train/weights/best.pt, 22.6MB

Validating /home/dw/ws_job_msislab/Golf_Project/runs_yolo/normal_train/weights/best.pt...
Ultralytics 8.3.175 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
Model summary (fused): 72 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 60/60 [00:02<00:00, 24.41it/s]


                   all        475        773      0.865      0.538      0.633      0.522
                 DIVOT        191        272      0.771       0.36      0.383       0.32
           FIXED_DIVOT        292        476      0.894      0.718      0.853       0.72
                 GRASS         25         25       0.93      0.534      0.664      0.525
Speed: 0.3ms preprocess, 3.6ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /home/dw/ws_job_msislab/Golf_Project/runs_yolo/normal_train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x75314950eb90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [11]:
from ultralytics import YOLO
from pathlib import Path

DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8/data.yaml"
RUN_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/runs_yolo/normal_train")

# 학습된 가중치 로드 (best.pt 권장)
model = YOLO(str(RUN_DIR / "weights" / "best.pt"))

# 검증 실행 (val 세트 기준)
r = model.val(
    data=DATA_YAML,
    split='val',        # 'val' 또는 'test'로 변경 가능
    imgsz=896,
    device=0,
    verbose=True,
    plots=False,        # 필요하면 True로
)

# === 전체 지표 ===
print(f"mAP@50-95 (전체): {r.box.map:.4f}")
print(f"mAP@50    (전체): {r.box.map50:.4f}")

# === 클래스별 AP (기본: AP@50-95) ===
# r.box.maps: 각 클래스별 AP@50-95 리스트 (len == 클래스 수)
# r.names: 클래스 id -> 이름 매핑(dict)
per_class_ap = r.box.maps
for cid, ap in enumerate(per_class_ap):
    print(f"{r.names[cid]:<20s} AP@50-95: {ap:.4f}")


Ultralytics 8.3.175 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
Model summary (fused): 72 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2080.7±61.3 MB/s, size: 3251.5 KB)


val: Scanning /home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8/labels/val.cache... 475 images, 63 backgrounds, 0 corrupt: 100%|██████████| 475/475 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:04<00:00,  6.40it/s]

                   all        475        773      0.865      0.537      0.633      0.521
                 DIVOT        191        272      0.769       0.36      0.382      0.319
           FIXED_DIVOT        292        476      0.895      0.718      0.853      0.723
                 GRASS         25         25       0.93      0.533      0.664       0.52
Speed: 0.3ms preprocess, 7.7ms inference, 0.0ms loss, 0.3ms postprocess per image


mAP@50-95 (전체): 0.5208
mAP@50    (전체): 0.6330
DIVOT                AP@50-95: 0.3192
FIXED_DIVOT          AP@50-95: 0.7234
GRASS                AP@50-95: 0.5199


In [12]:
import random, cv2, glob, os
val_img_dir = "/home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8/images/val"
val_lbl_dir = "/home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8/labels/val"
imgs = glob.glob(os.path.join(val_img_dir, "*.jpg"))

def show_divot(n=50):
    picks = []
    for p in imgs:
        y = os.path.join(val_lbl_dir, os.path.basename(p).replace(".jpg",".txt"))
        if not os.path.exists(y): continue
        with open(y) as f:
            for line in f:
                cid = int(line.split()[0])
                if cid==0:  # DIVOT
                    picks.append(p); break
    random.shuffle(picks)
    return picks[:n]

print("\n".join(show_divot(50)))

/home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8/images/val/Data_20250721_20250721_053640_0023_000073_f000720_1920x1536_2106.jpg
/home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8/images/val/Data_20250725_20250725_170115_0001_000011_f000300_1920x1536.jpg
/home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8/images/val/Data_20250721_20250721_053640_0031_000167_f001660_1920x1536_3684.jpg
/home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8/images/val/Data_20250721_20250721_053640_0007_000017_f000160_1920x1536_504.jpg
/home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8/images/val/Data_20250721_20250721_053640_0031_000035_f000340_1920x1536_3323.jpg
/home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8/images/val/Data_20250721_20250721_053640_0006_000129_f001280_1920x1536_323.jpg
/home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8/images/val/Data_20250721_20250721_053640_0023_000157_f001560_1920x1536_2230.jpg
/home/dw/ws_job_msis